In [1]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

%matplotlib inline

In [2]:
import os, sys
PARENT_DIR = "/home/Developer/NCSN-TF2.0/"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
sys.path.append(PARENT_DIR)

import PIL
import utils, configs
import argparse
import seaborn as sns
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
import matplotlib.pyplot as plt

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from helper import plot_confusion_matrix, metrics

from datasets.dataset_loader import  *
from tqdm import tqdm
from sklearn.metrics import classification_report, average_precision_score
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc

from sklearn.mixture import GaussianMixture
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from PIL import Image
from IPython.display import display
from matplotlib.pyplot import imshow
from datetime import datetime

import seaborn as sns
import plotly as py
import plotly.graph_objs as go

import umap
from sklearn.preprocessing import MinMaxScaler 

sns.set(style="darkgrid")
plt.rcParams['axes.titlesize'] = 18
plt.rcParams['axes.labelsize'] = 16

seed=42
tf.random.set_seed(seed)
np.random.seed(seed)

tf.__version__

'2.2.0'

In [3]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [4]:
from ood_detection_helper import *

In [5]:
LABELS = np.array(["Train", "CIFAR", "CelebA", "SVHN"])
colors = ["red", "blue", "green", "orange"]

In [6]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.labelsize'] = 18

## Resuming from latest checkpoint

In [7]:
model = load_model(inlier_name="svhn_cropped", save_path=PARENT_DIR+"saved_models/", checkpoint=-1)

(1, 32, 32, 3)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
images (InputLayer)             [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 128)  3584        images[0][0]                     
__________________________________________________________________________________________________
idx_sigmas (InputLayer)         [(None,)]            0                                            
__________________________________________________________________________________________________
conditional_full_pre_activation (None, 32, 32, 128)  302848      conv2d[0][0]                     
                                                                 idx_sigmas[0][

In [8]:
%%time

TEST_BATCH = 1000
svhn_train_batches = []

with tf.device('CPU'):
    data_generators = tfds.load(name="svhn_cropped", batch_size=-1, data_dir=PARENT_DIR+"data", shuffle_files=True)
    svhn_train = tf.data.Dataset.from_tensor_slices(data_generators['train']["image"]).take(73000)
    svhn_train = svhn_train.map(lambda x: x/255, num_parallel_calls=AUTOTUNE)
    svhn_train = svhn_train.batch(TEST_BATCH)
    
svhn_train_batches = [x for x in svhn_train]

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Unable to locate the source code of <function <lambda> at 0x7f1a81847510>. Note that functions defined in certain environments, like the interactive Python shell do not expose their source code. If that is the case, you should to define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Unable to locate the source code of <function <lambda> at 0x7f1a81847510>. Note that functions defined in certain environments, like the interactive Python shell do not expose their source code. If that is the case, you should to define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Unable to locate the source code of <function <lambda> at 0x7f1a81847510>. Note that functions defined in certain environments, like the interactive Python shell do not expose their source code. If that is the case, you should to define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
CPU times: user 4min 13s, sys: 2min 54s, total: 7min 8s
Wall time: 29.1 s


In [9]:
# def compute_score_norms(model, x_test):
    
#     # Sigma Idx -> Score
#     norm_batches = []
    
#     sigmas = utils.get_sigma_levels().numpy()
#     final_logits = 0 #tf.zeros(logits_shape)
#     progress_bar = tqdm(x_test)
    
#     for x_batch in x_test:
# #         progress_bar.set_description("Sigma: {:.4f}".format(sigma))
#         _logits = []
#         for idx, sigma in enumerate(sigmas):
#             idx_sigmas = tf.ones(x_batch.shape[0], dtype=tf.int32) * idx
#             score = model([x_batch, idx_sigmas])
#             _logits.append(score)

#         _logits = tf.concat(_logits, axis=0)
#         norm_batches.append(weighted_norm(_logits))
#         del _logits

#     return norm_batches

In [10]:
%%time
svhn_train_scores = compute_scores(model, svhn_train_batches)

Sigma: 0.0100: 100%|██████████| 10/10 [22:35<00:00, 135.51s/it]

CPU times: user 6min 47s, sys: 5min 51s, total: 12min 38s
Wall time: 22min 35s


In [13]:
svhn_train_norms = weighted_norm(svhn_train_scores)

In [14]:
svhn_train_norms.shape

TensorShape([73000, 10])

In [15]:
import pickle
with open("train_scores_ckpt-last.p", "wb") as f:
    pickle.dump(svhn_train_norms.numpy(), f)